In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
from dateutil.relativedelta import relativedelta

external_stylesheets = ['assets/style.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1("Client Subscription Management"),
    html.Div([
        html.Label("Client Name"),
        dcc.Input(id="client-name", type="text", placeholder="Enter client name"),
    ]),
    html.Div([
        html.Label("Payment Amount"),
        dcc.Input(id="payment-amount", type="number", placeholder="Enter payment amount"),
    ]),
    html.Div([
        html.Label("Subscription Date"),
        dcc.DatePickerSingle(
            id="subscription-date",
            placeholder="Select subscription date",
            display_format="YYYY-MM-DD"
        )
    ]),
    html.Div([
        html.Label("Subscription Duration"),
        dcc.Dropdown(
            id="subscription-duration",
            options=[
                {"label": "1 Month", "value": 1},
                {"label": "3 Months", "value": 3},
                {"label": "6 Months", "value": 6},
                {"label": "1 Year", "value": 12}
            ],
            placeholder="Select subscription duration"
        )
    ]),
    html.Div([
        html.Label("Client Type"),
        dcc.Dropdown(
            id="client-type",
            options=[
                {"label": "Personal", "value": "Personal"},
                {"label": "Consultoria", "value": "Consultoria"}
            ],
            placeholder="Select client type"
        )
    ]),
    html.Div([
        html.Button("Add Client", id="add-client-button", n_clicks=0),
        html.Span(style={"width": "10px"}),  # Adding space between buttons
        html.Button("Export to Excel", id="export-button", n_clicks=0),
    ]),
    html.Table([
        html.Thead(html.Tr([html.Th("Client Name"), html.Th("Payment Amount"), html.Th("Subscription Date"), html.Th("Subscription Duration"), html.Th("Client Type"), html.Th("Next Payment Date")])),
        html.Tbody(id="client-table")
    ]),
])

clients_df = pd.DataFrame(columns=["Client Name", "Payment Amount", "Subscription Date", "Subscription Duration", "Client Type", "Next Payment Date"])

@app.callback(
    Output("client-table", "children"),
    [Input("add-client-button", "n_clicks")],
    [State("client-name", "value"), State("payment-amount", "value"), State("subscription-date", "date"), State("subscription-duration", "value"), State("client-type", "value")]
)
def add_client(n_clicks, name, payment, subscription_date, duration, client_type):
    if n_clicks > 0:
        global clients_df
        subscription_start_date = pd.to_datetime(subscription_date).date()
        next_payment_date = subscription_start_date + relativedelta(months=duration)
        clients_df = clients_df.append({
            "Client Name": name,
            "Payment Amount": payment,
            "Subscription Date": subscription_start_date,
            "Subscription Duration": duration,
            "Client Type": client_type,
            "Next Payment Date": next_payment_date
        }, ignore_index=True)
    return [
        html.Tr(
            [
                html.Td(client["Client Name"]),
                html.Td(client["Payment Amount"]),
                html.Td(client["Subscription Date"]),
                html.Td(client["Subscription Duration"]),
                html.Td(client["Client Type"]),
                html.Td(
                    client["Next Payment Date"].strftime("%Y-%m-%d"),
                    style={"color": "red" if client["Next Payment Date"] < pd.Timestamp.today().date() else ""}
                ),
            ]
        )
        for _, client in clients_df.iterrows()
    ]

@app.callback(
    Output("export-button", "children"),
    [Input("export-button", "n_clicks")]
)
def export_to_excel(n_clicks):
    if n_clicks > 0:
        global clients_df
        clients_df.to_excel("client_data.xlsx", index=False)
        return "Data exported!"
    return "Export to Excel"

if __name__ == "__main__":
    app.run_server(debug='')


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/May/2023 21:50:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 21:50:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 21:50:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 21:50:24] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 21:50:24] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 21:50:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 21:50:24] "POST /_dash-update-component HTTP/1.1" 200 -
C:\Users\gnmrq\AppData\Local\Temp\ipykernel_16960\62909706.py:76: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

127.0.0.1 - - [25/May/2023 21:50:44] "POST /_dash-update-component HTTP/1.1" 200 -
C:\Users\gnmrq\AppData\Local\Temp\ipykernel_16960\62909706.p